In [1]:
#!accelerate launch --mixed_precision fp16 ./train_accel.py --config_name EXP_02

In [2]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V1

In [3]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V2

In [4]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V0

In [5]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V1

In [6]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V2

In [7]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPH

In [8]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPHV2

In [9]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V3

In [10]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V5

In [11]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V5

In [12]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V6

In [13]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V7

In [14]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V8

In [15]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V9

In [16]:
#!CUDA_VISIBLE_DEVICES=1 python eval.py --config_name BASELINE_HF_V8FTEVAL

In [17]:
# def set_gpu_environ():
#     """Sets CUDA_VISIBLE_DEVICES to those under minimal memory load.
#     Meant to be used in notebooks only.
#     """
#     import os
#     import subprocess
#     query = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used', '--format=csv']).decode().split('\n')[1:-1]
#     utilization = [int(x.replace(" MiB", "")) for x in query]
#     free = [i for i in range(len(utilization)) if utilization[i] == min(utilization)]
#     set_visible = ",".join([str(i) for i in free])
#     os.environ["CUDA_VISIBLE_DEVICES"] = set_visible
# set_gpu_environ()

In [18]:
import os
import random
import numpy as np
import torch
import config

def set_gpu_environ():
    """Sets CUDA_VISIBLE_DEVICES to those under minimal memory load.
    Meant to be used in notebooks only.
    """
    import os
    import subprocess
    query = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used', '--format=csv']).decode().split('\n')[1:-1]
    utilization = [int(x.replace(" MiB", "")) for x in query]
    free = [i for i in range(len(utilization)) if utilization[i] == min(utilization)]
    set_visible = ",".join([str(i) for i in free])
    os.environ["CUDA_VISIBLE_DEVICES"] = set_visible
    print(set_visible)
set_gpu_environ()

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-02-27 22:41:45 - get_logger - Writing log to logs/graphnet_20230227-224145.log
0,1


In [19]:
def seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def train(cfg):
    seed()
    custom_model = cfg.MODEL_NAME()
    if cfg.MODEL_WTS:
        print(f"Loading model weights from {cfg.MODEL_WTS}")
        custom_model.load_state_dict(torch.load(cfg.MODEL_WTS))
    opt = cfg.OPT(
        custom_model.parameters(), lr=cfg.LR, weight_decay=cfg.WD
    )
    loss_func = cfg.LOSS_FUNC()
    len_trn_dl = (cfg.N_FILES * 200000)//cfg.BATCH_SIZE
    warmup_steps = int(len_trn_dl * cfg.WARM_UP_PCT * cfg.EPOCHS)
    total_steps = int(len_trn_dl * cfg.EPOCHS)

    print(f"Total steps: {total_steps}, Warmup steps: {warmup_steps}")

    scheduler = cfg.SCHEDULER(
        opt,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    cfg.FIT_FUNC(
        epochs=cfg.EPOCHS,
        model=custom_model,
        loss_fn=loss_func,
        opt=opt,
        metric=cfg.METRIC,
        config = cfg,
        folder=cfg.FOLDER/cfg.EXP_NAME,
        exp_name=f"{cfg.EXP_NAME}",
        device=cfg.DEVICE,
        sched=scheduler,
    )
    
def main(config_name):
    configs = eval(f"config.{config_name}")
    print(f"Training with config: {configs.__dict__}")
    os.makedirs(configs.FOLDER/configs.EXP_NAME)
    train(configs)


In [ ]:
main('BASELINE_graph_V9_FT_2')

Training with config: {'__module__': 'config', 'EXP_NAME': 'EXP_34_FT_2', 'FILTER': False, 'MODEL_WTS': '/opt/slh/icecube/RESULTS/EXP_34_FT/EXP_34_FT_5.pth', 'LR': 0.0003, 'EPOCHS': 24, 'LOSS_FUNC': <class 'icecube.modelsgraph.gVonMisesFisher3DLoss'>, 'N_FILES': 50, 'TRN_BATCH_RANGE': [[1, 51], [51, 101], [101, 151], [151, 201], [201, 251], [251, 301], [301, 351], [351, 401], [401, 451], [451, 501], [501, 551], [501, 601]], 'DEVICE': 'cuda:1', 'NUM_WORKERS': 22, 'BATCH_SIZE': 1280, '__doc__': None}
Loading model weights from /opt/slh/icecube/RESULTS/EXP_34_FT/EXP_34_FT_5.pth
Total steps: 187488, Warmup steps: 18748


wandb: Currently logged in as: drhb (kaggle-hi). Use `wandb login --relogin` to force relogin


epoch,train_loss,valid_loss,val_metric


trn_range: [1, 51]
Better model found at epoch 0 with value: 1.012536883354187.
   epoch  train_loss  valid_loss     metric
0      0    1.469686      1.4786  1.0125369
trn_range: [51, 101]
Better model found at epoch 1 with value: 1.0128681659698486.
   epoch  train_loss  valid_loss     metric
0      1    1.477044     1.47958  1.0128682
trn_range: [101, 151]
Better model found at epoch 2 with value: 1.0168836116790771.
   epoch  train_loss  valid_loss     metric
0      2    1.487331    1.487236  1.0168836
trn_range: [151, 201]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 5 with value: 1.0167895555496216.
   epoch  train_loss  valid_loss     metric
0      5    1.484505      1.4865  1.0167896
trn_range: [301, 351]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 10 with value: 1.014186143875122.
   epoch  train_loss  valid_loss     metric
0     10    1.469914    1.469508  1.0141861
trn_range: [501, 601]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
torch.cuda.current_device()

In [ ]:
k

In [ ]:
kff

In [ ]:
k